# Подготовка данных

## Импорт библиотек и константы

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy import sparse
from collections import Counter
from random import randint, random
from rectools.columns import Columns
from rectools.dataset import Dataset
from sklearn.model_selection import train_test_split

In [3]:
INTERACTIONS_PATH = "./data/interactions.csv"
ITEMS_PATH = "./data/items.csv"
USERS_PATH = "./data/users.csv"
K_RECOMMENDATIONS = 10
RANDOM_STATE = 0

## Загрузка данных

In [4]:
interactions_df = pd.read_csv(INTERACTIONS_PATH)
interactions_df = interactions_df.rename(columns={"watched_pct": Columns.Weight, "last_watch_dt": Columns.Datetime})
interactions_df['datetime'] = pd.to_datetime(interactions_df['datetime'])
interactions_df.head(5)

,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [5]:
items_df = pd.read_csv(ITEMS_PATH)
items_df.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [6]:
users_df = pd.read_csv(USERS_PATH)
users_df.head(5)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


## Заполнение пропусков в данных и приведение типов

### Юзеры

In [7]:
users_df['age'] = users_df['age'].fillna('age_unknown')
users_df['age'] = users_df['age'].astype('category')
users_df['age'].value_counts()

age_25_34      233926
age_35_44      207043
age_45_54      135925
age_18_24      127672
age_55_64       75265
age_65_inf      46271
age_unknown     14095
Name: age, dtype: int64

In [8]:
users_df['income'] = users_df['income'].fillna('income_unknown')
users_df['income'] = users_df['income'].astype('category')
users_df['income'].value_counts()

income_20_40      471519
income_40_60      248330
income_60_90       68674
income_0_20        21836
income_unknown     14776
income_90_150      13985
income_150_inf      1077
Name: income, dtype: int64

In [9]:
users_df['sex'] = users_df['sex'].fillna('sex_unknown')
users_df.loc[users_df.sex == 'М', 'sex'] = 'sex_M'
users_df.loc[users_df.sex == 'Ж', 'sex'] = 'sex_F'
users_df['sex'] = users_df['sex'].astype('category')
users_df['sex'].value_counts()

sex_F          425270
sex_M          401096
sex_unknown     13831
Name: sex, dtype: int64

In [10]:
users_df['kids_flg'] = users_df['kids_flg'].astype('category')
users_df['kids_flg'] = users_df['kids_flg'].apply(lambda x: f"kids_flg_{x}")
users_df['kids_flg'].value_counts(dropna=False)

kids_flg_0    587209
kids_flg_1    252988
Name: kids_flg, dtype: int64

In [11]:
users_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840197 entries, 0 to 840196
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   user_id   840197 non-null  int64   
 1   age       840197 non-null  category
 2   income    840197 non-null  category
 3   sex       840197 non-null  category
 4   kids_flg  840197 non-null  category
dtypes: category(4), int64(1)
memory usage: 9.6 MB


### Айтемы

In [12]:
# В основном Тренировки от Motify и Лига чемпионов AFC, относящиеся к 2020 году
# Остальная часть относится к 2021 году, но мы потом переведем к категории "2020_inf", поэтому можно и просто 2020 год указать
items_df.loc[items_df['release_year'].isna(), 'release_year'] = 2020.

In [13]:
items_df.loc[items_df['release_year'] < 1920, 'release_year_cat'] = 'inf_1920'
items_df.loc[items_df['release_year'] >= 2020, 'release_year_cat'] = '2020_inf'

for i in range (1920, 2020, 10):
    items_df.loc[(items_df['release_year'] >= i) & (items_df['release_year'] < i+10), 'release_year_cat'] = f'{i}-{i+10}'

items_df = items_df.drop(columns=['release_year'])
items_df['release_year_cat'] = items_df['release_year_cat'].astype('category')

items_df.release_year_cat.value_counts()

2010-2020    8788
2020_inf     2276
2000-2010    2168
1980-1990     745
1990-2000     636
1970-1980     553
1960-1970     342
1950-1960     199
1940-1950     117
1930-1940     104
1920-1930      24
inf_1920       11
Name: release_year_cat, dtype: int64

In [14]:
# Тренировки от Motify 
items_df.loc[items_df.countries.isna(), 'countries'] = 'Россия'
items_df['countries'] = items_df['countries'].str.lower()
items_df['countries'] = items_df['countries'].apply(lambda x: ', '.join(sorted(list(set(x.split(', '))))))
items_df['countries'] = items_df['countries'].astype('category')
items_df['countries'].value_counts().head(10)

россия                 4274
сша                    4090
ссср                   1401
франция                1158
великобритания          718
украина                 340
италия                  256
канада                  233
республика корея        230
великобритания, сша     193
Name: countries, dtype: int64

In [15]:
# Машины-помощники и БиБаБу, которые являются детскими
items_df.loc[items_df.age_rating.isna(), 'age_rating'] = 0
items_df['age_rating'] = items_df['age_rating'].astype('category')
items_df['age_rating'].value_counts()

16.0    5729
12.0    4147
18.0    2547
6.0     1538
0.0     1520
21.0     482
Name: age_rating, dtype: int64

In [16]:
# считаем, что айтемы для детей это айтемы с возрастным ограниченим до 12+ лет
items_df.loc[items_df['for_kids'].isna(), "for_kids"] = items_df[items_df['for_kids'].isna()]['age_rating'].isin([0, 6, 12])
items_df['for_kids'] = items_df['for_kids'].astype('bool')
items_df['for_kids'].value_counts()

False    8861
True     7102
Name: for_kids, dtype: int64

In [17]:
items_df['genres'] = items_df['genres'].fillna('unknown')
items_df['genres'] = items_df['genres'].str.lower()
items_df['genres'] = items_df['genres'].apply(lambda x: ', '.join(sorted(list(set(x.split(', '))))))
items_df['genres'] = items_df['genres'].astype('category')
items_df['genres'].value_counts().head()

документальное        816
драмы                 719
комедии               564
для взрослых          482
мелодрамы, русские    416
Name: genres, dtype: int64

In [18]:
items_df['studios'] = items_df['studios'].fillna('unknown')
items_df['studios'] = items_df['studios'].str.lower()
items_df['studios'] = items_df['studios'].apply(lambda x: ', '.join(sorted(list(set(x.split(', '))))))
items_df['studios'] = items_df['studios'].astype('category')
items_df['studios'].value_counts().head()

unknown          14898
hbo                353
ленфильм           212
sony pictures      162
paramount           46
Name: studios, dtype: int64

In [19]:
items_df['directors'] = items_df['directors'].fillna('unknown')
items_df['directors'] = items_df['directors'].str.lower()
items_df['directors'] = items_df['directors'].astype('category')
items_df['directors'].value_counts().head()

unknown                1509
оливье шиабоду          511
денис франческо          49
глова роман юрьевич      36
сергей зарев             29
Name: directors, dtype: int64

In [20]:
items_df['actors'] = items_df['actors'].fillna('unknown')
items_df['actors'] = items_df['actors'].str.lower()
items_df['actors'] = items_df['actors'].astype('category')
items_df['actors'].value_counts().head()

unknown                                     2619
александр клюквин                             25
тайлер никсон, маркус лондон, томми ганн      16
жан-пьер морель, оливье шиабоду                8
кэти морган                                    6
Name: actors, dtype: int64

In [21]:
items_df['keywords'] = items_df['keywords'].fillna('unknown')
items_df['keywords'] = items_df['keywords'].str.lower()
items_df['keywords'] = items_df['keywords'].astype('category')
items_df['keywords'].value_counts().head()

unknown                        423
2015, чехия, компания, трех      6
2015, чехия, секс, страсть       6
2011, италия, дом, желаний       5
2006, италия, эвротико           5
Name: keywords, dtype: int64

In [22]:
items_df['description'] = items_df['description'].fillna('-')
items_df['description'] = items_df['description'].str.lower()
items_df['description'].head()

0    мелодрама легендарного педро альмодовара «пого...
1    уморительная современная комедия на популярную...
2    профессиональный рестлер стив остин («все или ...
3    шарлотта рэмплинг, том кортни, джеральдин джей...
4    расчетливая чаровница из советского кинохита «...
Name: description, dtype: object

In [23]:
items_df = items_df.drop(columns=["title_orig"])

In [24]:
items_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15963 entries, 0 to 15962
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   item_id           15963 non-null  int64   
 1   content_type      15963 non-null  object  
 2   title             15963 non-null  object  
 3   genres            15963 non-null  category
 4   countries         15963 non-null  category
 5   for_kids          15963 non-null  bool    
 6   age_rating        15963 non-null  category
 7   studios           15963 non-null  category
 8   directors         15963 non-null  category
 9   actors            15963 non-null  category
 10  description       15963 non-null  object  
 11  keywords          15963 non-null  category
 12  release_year_cat  15963 non-null  category
dtypes: bool(1), category(8), int64(1), object(3)
memory usage: 2.3+ MB


### Интеракции

In [25]:
interactions_df['weight'] = interactions_df['weight'].fillna(0)

In [26]:
interactions_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   user_id    5476251 non-null  int64         
 1   item_id    5476251 non-null  int64         
 2   datetime   5476251 non-null  datetime64[ns]
 3   total_dur  5476251 non-null  int64         
 4   weight     5476251 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


## Подготовка категориальных фичей

### Юзеры

In [27]:
user_cat_features = ["age", "income", "sex", "kids_flg"]
user_cat_encoding = dict()
for col in user_cat_features:
    cat_col = users_df[col].astype('category').cat
    user_cat_encoding[col] = cat_col.categories
    users_df[col] = cat_col.codes.astype('category')

In [28]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,1,4,1,1
1,962099,0,2,1,0
2,1047345,3,3,0,0
3,721985,3,2,0,0
4,704055,2,4,0,0


In [29]:
user_cat_dict = dict()
for key in user_cat_encoding:
    user_cat_dict[key] = dict()
    labels = user_cat_encoding[key]
    for (ind, label) in enumerate(labels):
        user_cat_dict[key][ind] = label

In [30]:
user_cat_dict

{'age': {0: 'age_18_24',
  1: 'age_25_34',
  2: 'age_35_44',
  3: 'age_45_54',
  4: 'age_55_64',
  5: 'age_65_inf',
  6: 'age_unknown'},
 'income': {0: 'income_0_20',
  1: 'income_150_inf',
  2: 'income_20_40',
  3: 'income_40_60',
  4: 'income_60_90',
  5: 'income_90_150',
  6: 'income_unknown'},
 'sex': {0: 'sex_F', 1: 'sex_M', 2: 'sex_unknown'},
 'kids_flg': {0: 'kids_flg_0', 1: 'kids_flg_1'}}

### Айтемы

In [31]:
item_cat_features = ['content_type', 'release_year_cat', 'for_kids', 'age_rating']
item_other_features = ['genres', 'countries', 'studios', 'actors', 'description']

In [32]:
item_cat_encoding = dict()
for col in item_cat_features:
    cat_col = items_df[col].astype('category').cat
    item_cat_encoding[col] = cat_col.categories
    items_df[col] = cat_col.codes.astype('category')

items_df = items_df[["item_id"] + item_cat_features + item_other_features]

In [33]:
items_df.head()

,item_id,content_type,release_year_cat,for_kids,age_rating,genres,countries,studios,actors,description
0,10711,0,8,0,3,"детективы, драмы, зарубежные, мелодрамы",испания,unknown,"адольфо фернандес, ана фернандес, дарио гранди...",мелодрама легендарного педро альмодовара «пого...
1,2508,0,9,0,3,"зарубежные, комедии, приключения",сша,unknown,"адам палли, брайан хаски, дж.б. смув, джейсон ...",уморительная современная комедия на популярную...
2,10716,0,9,0,3,"боевики, зарубежные, комедии, криминал, триллеры",канада,unknown,"адриан холмс, даррен шалави, джерри вассерман,...",профессиональный рестлер стив остин («все или ...
3,7868,0,9,0,3,"драмы, зарубежные, мелодрамы",великобритания,unknown,"александра риддлстон-барретт, джеральдин джейм...","шарлотта рэмплинг, том кортни, джеральдин джей..."
4,16268,0,5,1,2,"драмы, мелодрамы, советские, спорт",ссср,ленфильм,"александр абдулов, александр демьяненко, алекс...",расчетливая чаровница из советского кинохита «...


In [34]:
item_cat_dict = dict()
for key in item_cat_encoding:
    item_cat_dict[key] = dict()
    labels = item_cat_encoding[key]
    for (ind, label) in enumerate(labels):
        item_cat_dict[key][ind] = label

In [35]:
item_cat_dict

{'content_type': {0: 'film', 1: 'series'},
 'release_year_cat': {0: '1920-1930',
  1: '1930-1940',
  2: '1940-1950',
  3: '1950-1960',
  4: '1960-1970',
  5: '1970-1980',
  6: '1980-1990',
  7: '1990-2000',
  8: '2000-2010',
  9: '2010-2020',
  10: '2020_inf',
  11: 'inf_1920'},
 'for_kids': {0: False, 1: True},
 'age_rating': {0: 0.0, 1: 6.0, 2: 12.0, 3: 16.0, 4: 18.0, 5: 21.0}}

## Разбиение датасета

In [36]:
max_date = interactions_df[Columns.Datetime].max()
min_date = interactions_df[Columns.Datetime].min()
print(f'Продолжительность: {max_date - min_date}')

Продолжительность: 162 days 00:00:00


In [37]:
RANKER_DAYS_COUNT = 31
RANKER_TRAIN_SIZE = 0.7
RANKER_VALID_SIZE = 0.15
RANKER_TEST_SIZE = 0.15

Разделим датасет на две части по времени: для обучения моделей первого уровня и для обучения и тестирования ранкера

In [38]:
ranker_interactions_df = interactions_df[
    (interactions_df[Columns.Datetime] >= max_date - pd.Timedelta(days=RANKER_DAYS_COUNT))
]

base_models_interactions_df = interactions_df[
    (interactions_df[Columns.Datetime] < max_date - pd.Timedelta(days=RANKER_DAYS_COUNT))
]

Разделим пользователей из выборки для ранкера на три части: тренировочных, валидационных, тестовых

In [39]:
train_users, test_and_valid_users = train_test_split(
    ranker_interactions_df[Columns.User].unique(), random_state=RANDOM_STATE, test_size=(RANKER_VALID_SIZE + RANKER_TEST_SIZE)
)
valid_users, test_users = train_test_split(
    test_and_valid_users, random_state=RANDOM_STATE, test_size=RANKER_VALID_SIZE / (RANKER_VALID_SIZE + RANKER_TEST_SIZE)
)

In [40]:
len(train_users), len(valid_users), len(test_users)

(309324, 66284, 66284)

## Фильтрация

### Фильтрация айтемов с редкими интеракциями

In [41]:
item_popularities = interactions_df.groupby("item_id")["user_id"].count()
n_items = len(item_popularities)
more_1_n_items = len(item_popularities[item_popularities > 1])
more_5_n_items = len(item_popularities[item_popularities > 5])
more_10_n_items = len(item_popularities[item_popularities > 10])
more_25_n_items = len(item_popularities[item_popularities > 25])
more_50_n_items = len(item_popularities[item_popularities > 50])
more_100_n_items = len(item_popularities[item_popularities > 100])

print("Количество айтемов в интеракциях", n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 1 пользователя", more_1_n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 5 пользователей", more_5_n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 10 пользователей", more_10_n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 25 пользователей", more_25_n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 50 пользователей", more_50_n_items)
print("Количество айтемов в интеракциях, которые посмотрели больше 100 пользователей", more_100_n_items)

Количество айтемов в интеракциях 15706
Количество айтемов в интеракциях, которые посмотрели больше 1 пользователя 13449
Количество айтемов в интеракциях, которые посмотрели больше 5 пользователей 9829
Количество айтемов в интеракциях, которые посмотрели больше 10 пользователей 8175
Количество айтемов в интеракциях, которые посмотрели больше 25 пользователей 6217
Количество айтемов в интеракциях, которые посмотрели больше 50 пользователей 4962
Количество айтемов в интеракциях, которые посмотрели больше 100 пользователей 3894


In [42]:
k = 25
items_more_k_interactions = item_popularities[item_popularities > k].index
print(f"Доля интеракций после удаления айтемов, с которым взаимедействовали меньше {k} пользователей:", (
    round(len(base_models_interactions_df[base_models_interactions_df[Columns.Item].isin(items_more_k_interactions)]) 
    / len(base_models_interactions_df), 2)
))

Доля интеракций после удаления айтемов, с которым взаимедействовали меньше 25 пользователей: 0.99


Оставим только те интеракции в обучающей выборке с айтемами, с которыми взаимодействовали больше 25 пользователей

In [43]:
base_models_interactions_df = base_models_interactions_df[base_models_interactions_df[Columns.Item].isin(items_more_k_interactions)]

## Сохранение данных

In [44]:
mkdir new_data

mkdir: cannot create directory ‘new_data’: File exists


In [45]:
base_models_interactions_df.to_csv("new_data/base_models_data.csv", index=False)

In [46]:
users_df.to_csv("new_data/user_features_data.csv", index=False)

In [47]:
items_df.to_csv("new_data/item_features_data.csv", index=False)

In [48]:
ranker_interactions_df[ranker_interactions_df[Columns.User].isin(train_users)].to_csv("new_data/ranker_train_data.csv", index=False)

In [49]:
ranker_interactions_df[ranker_interactions_df[Columns.User].isin(valid_users)].to_csv("new_data/ranker_valid_data.csv", index=False)

In [50]:
ranker_interactions_df[ranker_interactions_df[Columns.User].isin(test_users)].to_csv("new_data/ranker_test_data.csv", index=False)

In [51]:
with open("new_data/item_features_mapping.json", "w", encoding="utf-8") as f:
    json.dump(item_cat_dict, f)

In [52]:
with open("new_data/user_features_mapping.json", "w", encoding="utf-8") as f:
    json.dump(user_cat_dict, f)